# Stacking:

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from sklearn import preprocessing
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold


sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



pd.set_option('display.max_columns', None)

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [35]:
from vecstack import stacking


In [36]:
train5 = pd.read_csv('data/train4.csv', index_col=False)

In [37]:
test5 = pd.read_csv('data/test4.csv', index_col=False)

In [38]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

In [39]:
import lightgbm as lgb


In [40]:
from catboost import Pool, CatBoostClassifier



In [41]:
models = [RandomForestClassifier(),\
          CatBoostClassifier(max_depth=12, n_estimators=1000, learning_rate=0.4, random_state=0, silent=True)]

In [42]:
train_with_index = train5.drop(['fraud','time1'], axis=1)
train_fraud_with_index = train5.fraud

In [43]:
X_train, X_test, y_train, y_test = train_test_split(train_with_index, train_fraud_with_index, random_state=0, test_size=0.2)

X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [44]:
S_train, S_test = stacking(models,X_train, y_train, X_test,regression=False, mode='oof_pred_bag', needs_proba=False,\
                           save_dir=None, metric=accuracy_score, n_folds=5, stratified=True, shuffle=True,\
                           random_state=0, verbose=2)
                              

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.99361770]
    fold  1:  [0.99387644]
    fold  2:  [0.99360127]
    fold  3:  [0.99369163]
    fold  4:  [0.99360536]
    ----
    MEAN:     [0.99367848] + [0.00010427]
    FULL:     [0.99367848]

model  1:     [CatBoostClassifier]
    fold  0:  [0.99411876]
    fold  1:  [0.99417215]
    fold  2:  [0.99402019]
    fold  3:  [0.99390109]
    fold  4:  [0.99397088]
    ----
    MEAN:     [0.99403661] + [0.00009806]
    FULL:     [0.99403661]



In [53]:
model_final = RandomForestClassifier()
    
model_final = model.fit(S_train, y_train)
y_pred = model_final.predict(S_test)

In [54]:
print("=== f1 score on test===")
print(f1_score(y_test, y_pred))

=== f1 score on test===
0.7872763419483101


In [50]:
test5 = pd.read_csv('data/test4.csv', index_col=False)

In [51]:
test_with_index = test5.drop(['time1'], axis=1)

In [55]:
model_predict = model_final.predict(test_with_index)

ValueError: Number of features of the model must match the input. Model n_features is 2 and input n_features is 28 

In [ ]:
test_with_index['fraud_ind'] = model_predict
test_with_index.fraud_ind.value_counts()

In [ ]:
test_with_index['fraud_ind'] = model_predict
test_with_index.fraud_ind.value_counts()
test_with_index['txkey']=test_with_index.transaction_no
model_best = test_with_index[['txkey','fraud_ind']]

In [ ]:
model_best.to_csv('data/stack.csv', index=False)